## mBART Pretraining with Hybrid Gaussian Random Embeddings for Low-Resource IGT

### Predefined function

In [ ]:

!pip uninstall -y numpy evaluate unbabel-comet rouge_score


!pip install numpy==1.23.5


!pip install evaluate unbabel-comet rouge_score

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 75.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.89 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.2.1 requi

  Preparing metadata (setup.py) ... done
^C


In [ ]:
!pip install --force-reinstall evaluate unbabel-comet

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached unbabel_comet-2.2.5-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-3.5.0-py3-none-any.whl.metadata (19 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.8 MB/s eta 0:00:00
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.7 MB/s eta 0:00:00
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.17-py311-none-any.whl.metadata (7.2 kB)
  Using cached entmax-1.3-py3-none-any.whl.metadata (348 bytes)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
from evaluate import load
from comet import download_model, load_from_checkpoint

def evaluate_mt_model(trainer, tokenizer, tokenized_test, raw_sources):
    print("🔍 Running model predictions...")
    preds = trainer.predict(tokenized_test)
    decoded_preds = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(preds.label_ids, skip_special_tokens=True)

    print("📏 Computing BLEU, ChrF++, ROUGE, and Exact Match...")
    bleu = load("sacrebleu")
    chrf = load("chrf")
    rouge = load("rouge")

    bleu_score = bleu.compute(predictions=decoded_preds,
                              references=[[ref] for ref in decoded_labels])["score"]
    chrf_score = chrf.compute(predictions=decoded_preds,
                              references=decoded_labels)["score"]
    rouge_score = rouge.compute(predictions=decoded_preds,
                                references=decoded_labels)["rougeL"]

    exact_matches = [pred.strip() == label.strip() for pred, label in zip(decoded_preds, decoded_labels)]
    exact_match_score = sum(exact_matches) / len(exact_matches) * 100

    print("🧠 Computing COMET score...")
    model_path = download_model("Unbabel/wmt22-comet-da")
    comet_model = load_from_checkpoint(model_path)

    comet_data = [
        {"src": raw_sources[i], "mt": decoded_preds[i], "ref": decoded_labels[i]}
        for i in range(len(decoded_preds))
    ]
    comet_score = comet_model.predict(comet_data, batch_size=8, gpus=1)
    comet_mean = np.mean(comet_score.scores)

    print("\n✅ Evaluation Summary:")
    print(f"BLEU:        {bleu_score:.2f}")
    print(f"ChrF++:      {chrf_score:.2f}")
    print(f"ROUGE-L:     {rouge_score:.2f}")
    print(f"Exact Match: {exact_match_score:.2f}%")
    print(f"COMET:       {comet_mean:.4f}")

    return {
        "BLEU": bleu_score,
        "ChrF++": chrf_score,
        "ROUGE-L": rouge_score,
        "Exact Match (%)": exact_match_score,
        "COMET": comet_mean
    }

### Install and Import Required Libraries

In [ ]:
!pip install transformers datasets sentencepiece accelerate

import torch
import json
import numpy as np
from datasets import load_dataset
from transformers import MBartTokenizer, MBartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

### Load Interlinear Glossed Text (IGT) Data
* Spanish, Gloss, and Kekchi triples

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/mbart_igt_pretrain_v2.jsonl", split="train")


dataset = dataset.train_test_split(test_size=0.1, seed=42)


train_data = dataset["train"]
test_data = dataset["test"]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data[:5]

{'input': ['[SRC] jesús dijo que el tabaco no es bueno para las personas [GLOSS] jesús jesus chi chi may chi chi us chi chi chi',
  '[SRC] cuanto más compasivo fiel y desinteresado sea nuestro servicio y sacrificio por él más podremos comenzar a comprender la compasión y gracia expiatoria de jesucristo por nosotros [GLOSS] jo chi a chi chi desinteresado chi chi chi chi chi chi chi chi chi chi chi chi chi compasión chi chi of chi jesucristo chi chi',
  '[SRC] aprenda cómo enviar sus himnos o canciones originales para que se consideren incluirlas en el nuevo himnario y canciones para los niños [GLOSS] jo chi laa chi chi chi jesus chi chi chi chi lee incluirlas chi chi chi himnario chi jesus chi chi chi',
  '[SRC] y habiendo creído que era la tierra de zarahemla ellos se volvieron a la tierra de nefi llegando a los confines del país no muchos días antes de la venida de ammón [GLOSS] chi chi we chi chi chi chi chi chi chi chi a chi chi chi chi nefi laa chi chi jo del chi chi chi chi chi ch

### Load mBART-50 Model and Tokenizer
* planning - modifying its output embeddings.

In [ ]:
model_name = "facebook/mbart-large-50"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

### Hybrid Embedding Injection (γ * pretrained + (1 - γ) * random)
* For each token, compute a weighted average of its pretrained vector and a random Gaussian vector, then normalize it.

* Use a fixed γ value (e.g., 0.9 as used in the paper)

In [ ]:
def inject_hybrid_embeddings(model, gamma=0.9):
    embedding = model.model.shared.weight.data
    vocab_size, dim = embedding.shape

    for idx in range(vocab_size):
        pretrained_vec = embedding[idx]
        rand_vec = torch.randn(dim)
        rand_vec /= rand_vec.norm()

        combined = gamma * pretrained_vec + (1 - gamma) * rand_vec
        combined /= combined.norm()
        embedding[idx] = combined

inject_hybrid_embeddings(model, gamma=0.7)

* This ensures that rare and frequent tokens are perturbed while preserving some semantic structure from the pretrained model.

### Preprocess IGT Dataset (Tokenize Inputs and Targets)

In [ ]:
def preprocess_function(examples):
    inputs = ["translate Spanish to Kekchi: " + x for x in examples["input"]]
    targets = examples["target"]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

Map:   0%|          | 0/148412 [00:00<?, ? examples/s]

Map:   0%|          | 0/16491 [00:00<?, ? examples/s]

### ALL Data

In [ ]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    max_steps=3000,
    eval_steps=500,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    evaluation_strategy="steps",
    predict_with_generate=True,
    fp16=True,
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test.select(range(1000)),
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model)
)

<ipython-input-11-99f341e38ac4>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.537400,0.482059
1000,0.407300,0.381203
1500,0.388700,0.339466
2000,0.367400,0.321610
2500,0.354900,0.324528
3000,0.331200,0.315427


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3000, training_loss=0.46426651255289714, metrics={'train_runtime': 1083.1873, 'train_samples_per_second': 88.627, 'train_steps_per_second': 2.77, 'total_flos': 2.6005557215232e+16, 'train_loss': 0.46426651255289714, 'epoch': 0.6468305304010349})

### Evaluation Score

In [ ]:
!pip install rouge_score

  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24986 sha256=62280a8a0a36478d4dfcbfcb6d84ef6723b210c0dc23bdfef71e265dac2adaa6
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
raw_sources = [
    x.split("[GLOSS]")[0].replace("[SRC]", "").strip()
    for x in test_data["input"]
]


evaluate_mt_model(trainer, tokenizer, tokenized_test.select(range(1000)), raw_sources[:1000])

🔍 Running model predictions...


📏 Computing BLEU, ChrF++, ROUGE, and Exact Match...


🧠 Computing COMET score...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model.ckpt:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_


✅ Evaluation Summary:
BLEU:        0.24
ChrF++:      5.46
ROUGE-L:     0.05
Exact Match: 0.00%
COMET:       0.2238


{'BLEU': 0.23814602165992227,
 'ChrF++': 5.463350810826553,
 'ROUGE-L': 0.04946932140568362,
 'Exact Match (%)': 0.0,
 'COMET': 0.22381501111388208}